In [1]:
# !pip3 install pandas_datareader
# !pip3 install pandas
# !pip3 install datetime 
# !pip3 install yahoo_fin 

import pandas_datareader as web ## USE TO LOAD DATA FROM WEBSITE
import pandas as pd
import numpy as np
import datetime as dt
import traceback
from yahoo_fin import stock_info as si


In [2]:
def breaker():
    print('X' * 50) 

## establish stock range 
start = dt.datetime.now()-dt.timedelta(days=365)
end = dt.datetime.now()



'''
    * 1. return s&p500 corp.  
    * 2. use DataReader to create sp500_df 
    * 3. create new col Pct Change and calculate percent change.  
    * 4. calculate YOY return for s&p 500, spanning all 500 companies. [cumprod()-1]
'''
stock_range = si.tickers_sp500() # 
sp500_df = web.DataReader('^GSPC', 'yahoo', start, end) 
sp500_df['Pct Change'] = sp500_df['Adj Close'].pct_change() 

sp500_return = (sp500_df['Pct Change'] + 1).cumprod()[-1] 

print(sp500_df.head()); print('X'*50)  
print(sp500_return)


                   High          Low         Open        Close      Volume  \
Date                                                                         
2021-01-14  3823.600098  3792.860107  3814.979980  3795.540039  5180140000   
2021-01-15  3788.729980  3749.620117  3788.729980  3768.250000  5353060000   
2021-01-19  3804.530029  3780.370117  3781.879883  3798.909912  4982940000   
2021-01-20  3859.750000  3816.219971  3816.219971  3851.850098  4551790000   
2021-01-21  3861.449951  3845.050049  3857.459961  3853.070068  4484460000   

              Adj Close  Pct Change  
Date                                 
2021-01-14  3795.540039         NaN  
2021-01-15  3768.250000   -0.007190  
2021-01-19  3798.909912    0.008136  
2021-01-20  3851.850098    0.013936  
2021-01-21  3853.070068    0.000317  
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
1.2275011558847984


In [3]:
return_list = []
final_df = pd.DataFrame(columns=['Ticker', 'Latest_Price', 'Score', 'PE_Ratio', 'PEG_Ratio', 'SMA_150', 'SMA_200', '52_Week_Low', '52_Week_High'])


''' 
    * Iterate through top performing org in s&p. [READ] In case, it is set to 10
    * Pull data for individual stocks and save to stockdata.csv
    * Simularly to S&P 500 Calculation, cal
    * Calculate and append pct change using 
    * Calculate stock return by dividing (pct_change + 1) by s&p500 return.  
    * save info to list (returns_list), which will contain date and %change per day 
'''
count = 0
for ticker in stock_range:
    df = web.DataReader(ticker, 'yahoo', start, end)
   # df['Ticker'] = ticker 
    df.to_csv(f"{ticker}.csv") # Write df Object to .CSV to access later. 
    
    df['Pct Change'] = df['Adj Close'].pct_change()
    stock_return = (df['Pct Change'] + 1).cumprod()[-1]
    
    returns_compared = round((stock_return / sp500_return), 2)
    return_list.append(returns_compared)
    count +=1
    if count == 30: break


print(df.head()); breaker()
print(stock_return)
print(type(stock_return)); breaker()
print(return_list) 



                 High        Low       Open      Close    Volume  Adj Close  \
Date                                                                          
2021-01-14  92.360001  90.449997  91.769997  90.790001  53408200  90.790001   
2021-01-15  91.589996  87.860001  90.750000  88.209999  49340300  88.209999   
2021-01-19  89.580002  87.239998  89.559998  89.449997  45115100  89.449997   
2021-01-20  90.779999  88.599998  90.550003  88.750000  35185800  88.750000   
2021-01-21  92.070000  88.430000  89.339996  91.529999  54221900  91.529999   

            Pct Change  
Date                    
2021-01-14         NaN  
2021-01-15   -0.028417  
2021-01-19    0.014057  
2021-01-20   -0.007826  
2021-01-21    0.031324  
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
1.4620553381937678
<class 'numpy.float64'>
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[0.94, 0.96, 1.18, 1.09, 1.01, 1.06, 0.79, 0.97, 1.17, 0.91, 0.88, 1.12, 1.19, 0.68, 1.0, 0.97, 0.71, 1.13, 1.25, 0.93, 1.16, 

In [4]:
'''
    * Best Performers is used by taking the previous 10 values from s&p and zipped 
    * Create a 'Score' by ranking 
'''

best_performers = pd.DataFrame(list(zip(stock_range, return_list)),
                               columns=['Ticker', 'Returns Compared'])

best_performers.dropna() 
best_performers['Score'] = best_performers['Returns Compared'].rank(pct=True) * 100
best_performers = best_performers[best_performers['Score'] >= best_performers['Score'].quantile(0.7)] #|[best_performers['Score'] <= best_performers['Score'].quantile(0.3)]


print(best_performers.head())
print(type(best_performers))


   Ticker  Returns Compared      Score
2     AAP              1.18  86.666667
8     ACN              1.17  83.333333
11    ADM              1.12  73.333333
12    ADP              1.19  91.666667
17    AFL              1.13  76.666667
<class 'pandas.core.frame.DataFrame'>


In [5]:
print(ticker)
print(df.columns) 
#print(si.get_stats_valuation('AAPL'))


for ticker in best_performers['Ticker']:
    try:
        df = pd.read_csv(f"{ticker}.csv", index_col=0)  ## load df from CSV.
        moving_averages = [150, 200]
        
        for ma in moving_averages:
            df['SMA_' + str(ma)] = round(df['Adj Close'].rolling(window=ma).mean(), 2) ## THIS IS LINKED TO MANY ERRORS BECAUSE NOT ROUNDED TO TWO 
            
            latest_price = df['Adj Close'][-1]
            pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
            
            
            #peg_ratio = float(si.get_stats_valuation(ticker)[1][4])
            
            moving_average_150 = df['SMA_150'][-1]
            #moving_average_200 = df['SMA_200'][-1]
            
            low_52week = round(min(df['Low'][-(52 * 5):]), 2)
            high_52week = round(min(df['High'][-(52 * 5):]), 2)

            score = round(best_performers[best_performers['Ticker'] == ticker]['Score'].tolist()[0])
            
            ## conditions to accept stock we are interested in
            #condition_1 = latest_price > moving_average_150 > moving_average_200
            condition_2 = latest_price >= (1.3 * low_52week)
            condition_3 = latest_price >= (.75 * high_52week)
            condition_4 = pe_ratio < 40
            #condition_5 = peg_ratio < 2

            ## final dataframe after conditons are met.
            if (condition_2 and condition_3 and condition_4): # OMMITTED condition_1 and condition_5
                final_df = final_df.append({'Ticker': ticker,
                                            'Latest_Price': latest_price,
                                            'Score': score,
                                            'PE_Ratio': pe_ratio,
                                            #'PEG_Ratio': peg_ratio,
                                            'SMA_150': moving_average_150,
                                            #'SMA_200': moving_average_200,
                                            '52_Week_Low': low_52week, 
                                            '52_Week_High': high_52week}, ignore_index=True)
                
                
    except Exception as e:
        print(f'[{ticker}] \n, {str(e)}  \n {traceback.format_exc()}')
        pass


AMD
Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'Pct Change'], dtype='object')


In [6]:

final_df.sort_values(by='Score', ascending=False)
pd.set_option('display.max_columns', 10)
print(final_df.head())
final_df.to_csv('final.csv')

  Ticker  Latest_Price Score  PE_Ratio  PEG_Ratio  SMA_150  SMA_200  \
0    AAP    241.320007    87     23.84        NaN   216.71      NaN   
1    AAP    241.320007    87     23.84        NaN   216.71      NaN   
2    ACN    361.369995    83     38.80        NaN   340.73      NaN   
3    ACN    361.369995    83     38.80        NaN   340.73      NaN   
4    ADM     70.900002    73     15.10        NaN    62.34      NaN   

   52_Week_Low  52_Week_High  
0       143.15        154.45  
1       143.15        154.45  
2       241.73        248.90  
3       241.73        248.90  
4        49.28         50.75  
